In [125]:
import urllib.request
import datetime
import json
import time
from datetime import datetime

import pandas as pd
import cx_Oracle

In [126]:
access_key="bgOnt78reFNsTUJuAwlI30JDObTxX6hbJCxyApJCtuf3xjJZJ%2FmOs8Vhg3GZAsLc1fXTkQ9sjq0mTEupWDdyyA%3D%3D"

In [127]:
def get_request_url(url):                 ## (1) 기상 정보(동네예보정보 조회 서비스)

    parameters = "?_type=json&serviceKey=" + access_key
    parameters += "&base_date=" + yyyymmdd
    parameters += "&base_time=" + day_time
    parameters += "&nx=" + x_coodinate
    parameters += "&ny=" + y_coodinate
    parameters += "&numOfRows=30"
    parameters += "&dataType=JSON"
    url = url + parameters

    req = urllib.request.Request(url)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.now(), url))
        return None

In [128]:
def json_to_df_info(raw_json):     ## (1) 기상 정보(동네예보정보 조회 서비스) json 파일 생성하는 함수

    all_data=[]
    column_list = ['baseDate', 'baseTime', 'category', 'nx', 'ny', 'obsrValue']
    for record in raw_json['response']['body']['items']['item']:
        # 이하 프로그램을 학생을이 작성하게 해볼것2
        row_data = []
        for column_data in column_list:
            row_data.append(record.get(column_data))
        all_data.append(row_data)

    return column_list, all_data

In [129]:
def get_update_time_info():        ## (1) 기상 정보(동네예보정보 조회 서비스) json 파일 만들기 전, 실시간 업데이트 확인 함수
    day_time = time.strftime("%H%M", time.localtime(time.time()))

    day_hour = time.strftime("%H")
    day_min = time.strftime("%M")

    day_min_int = int(day_min)
    if 30 <= day_min_int <= 59:      ## 실시간 업데이트가 있는지 없는지 확인,, 30분부터 59분까지는 실시간 정보 업데이트 됨
        day_time = time.strftime("%H%M")
        print("\n<<실시간 정보 업데이트를 실시합니다!!>>\n".center(30))

    elif 0 <= day_min_int < 30:        ## 실시간 업데이트가 되지 않을 경우, 가장 최신인 한 시간 전껄로
        day_hour_int = int(day_hour)
        day_hour_int = day_hour_int - 1
        revised_min = 60 + (day_min_int-30) # 시간은 60분 단위 이므로 30분 미만은 30분을 빼고 60을 더한다.
        day_time = "{0:0>2}".format(day_hour_int) + str(revised_min)      ## 시간이 한 자리 수일 때 930 되는 것을 0930으로 바꿔 줌

        print("\n<<가장 최신정보 업데이트를 실시합니다!!>>\n".center(30))
    return day_time

In [130]:
def preprocessed_df_to_oracle (df):
    sql_insert = '''
            insert into weather(DATE_TIME, NX, NY, 시간1_강수량, 강수형태, 기온, 습도, 풍향,풍속,동서바람성분,남북바람성분) 
            values(:DATE_TIME, :NX, :NY, :시간1_강수량, :강수형태, :기온, :습도, :풍향,:풍속,:동서바람성분,:남북바람성분)
            '''
    DATE_TIME = df.iloc[0]['DATE_TIME']
    NX = int(df.iloc[0]['NX']) # int 값에 대해서는 int 형으로 변환해줘야 한다.
    NY = int(df.iloc[0]['NY'])
    시간1_강수량 = df.iloc[0]['시간1_강수량']
    강수형태 = df.iloc[0]['강수형태']
    기온 = df.iloc[0]['기온']
    습도 = df.iloc[0]['습도']
    풍향 = df.iloc[0]['풍향']
    풍속 = df.iloc[0]['풍속']
    동서바람성분 = df.iloc[0]['동서바람성분']
    남북바람성분 = df.iloc[0]['남북바람성분']
    
    print('입력값')
    print(DATE_TIME, NX, NY, 시간1_강수량, 강수형태, 기온, 습도, 풍향, 풍속,동서바람성분, 남북바람성분)
    # 20221227 1600 58 125 0.0 0.0 1.9 54.0 307.0 1.5 1.2 -0.8
    print(type(DATE_TIME), type(NX), type(NY), type(시간1_강수량) ) #, 강수형태, 기온, 습도, 풍향, 풍속,동서바람성분, 남북바람성분)
    
    cur.execute(sql_insert,
                (
                    DATE_TIME,
                    NX,
                    NY,
                    시간1_강수량,
                    강수형태,
                    기온,
                    습도,
                    풍향,
                    풍속,
                    동서바람성분,
                    남북바람성분
                )
               )

    con.commit()

In [131]:
def preprocess_df (df):
    df.insert(0,'DateTime',df['baseDate']+df['baseTime'])
    nx = df.loc[0,'nx']
    ny = df.loc[0,'ny']
    date_time = df.loc[0, 'baseDate'] + ' ' + df.loc[0, 'baseTime']
    p_df = pd.pivot_table(df,index='DateTime',columns=['category'], values='obsrValue')
    p_df.insert(0,'date_time',[date_time])
    p_df.insert(1,'nx',[nx])
    p_df.insert(2,'ny',[ny])

    p_df.rename(columns={'date_time': 'DATE_TIME', 'nx': 'NX', 'ny': 'NY', 'PTY': '강수형태', 'REH': '습도',
                       'RN1': '시간1_강수량','T1H':'기온','UUU':'동서바람성분','VEC':'풍향','VVV':'남북바람성분','WSD':'풍속'}, inplace=True)
    redefined_columns = ['DATE_TIME','NX','NY','시간1_강수량','강수형태','기온','습도', '풍향','풍속','동서바람성분','남북바람성분']
    p_df = p_df[redefined_columns]
    return p_df

In [132]:
con = cx_Oracle.connect('open_source/1111@localhost:1521/xe')
cur = con.cursor()

In [133]:
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
# 업데이트는 30~60 사이에 이루어짐
yyyymmdd = time.strftime("%Y%m%d")
day_time = get_update_time_info()

# 구로동 좌표
x_coodinate = "58"
y_coodinate = "125"

raw_str_json = get_request_url(url)

 
<<가장 최신정보 업데이트를 실시합니다!!>>
  
[2022-12-27 17:18:15.264785] Url Request Success


In [134]:
if raw_str_json:
    raw_json = json.loads(raw_str_json)

In [135]:
column_list, all_data = json_to_df_info(raw_json)

df = pd.DataFrame(all_data, columns=column_list)

df_preprocessed = preprocess_df (df)

In [136]:
preprocessed_df_to_oracle(df_preprocessed)

입력값
20221227 1600 58 125 0.0 0.0 1.7 55.0 315.0 1.0 0.7 -0.6
<class 'str'> <class 'int'> <class 'int'> <class 'numpy.float64'>
